In [12]:
import pandas as pd
from functools import reduce
import numpy as np
import re
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import pprint
import app_credentials
import json
from pandas.io.json import json_normalize
pd.options.display.max_rows = 1000
billboard = pd.read_csv("./datasets/billboard100.csv")

In [13]:
def name_slicer(name):
    repls = {'X ' : 'Featuring', 'x ' : 'Featuring', '&' : 'Featuring', '+':'Featuring', ',':'Featuring', 'Feat.':'Featuring','With':'Featuring'}
    if name in ['Earth, Wind & Fire','Blood, Sweat & Tears']:
        pass
    else:
        name = reduce(lambda a, kv: a.replace(*kv), repls.items(), name)
        if 'Featuring' in name:
            name = name[:name.find("Featuring")-1]
    return name
def get_featuring(name):
    repls = {'X' : '|', 'x' : '|', '&' : '|', '+':'|', ',':'|', 'Feat.':'|','With':'|','Featuring':'|'}
    if name in ['Earth, Wind & Fire','Blood, Sweat & Tears']:
        pass
    else:
        name = reduce(lambda a, kv: a.replace(*kv), repls.items(), name)
        if '|' in name:
            name = name[2+name.find("|"):]
    return name
def get_Season(month):
    if month in (1,2,11,12):
        season = 'winter'
    elif month in (3,4,5):
        season = 'spring'
    elif month in (6,7,8):
        season = 'summer'
    else:
        season = 'autumn'
    return season

In [14]:
billboard.replace('-',np.nan,inplace=True)
billboard['artist1'] = billboard['artist'].apply(lambda x: name_slicer(x))
billboard['featuring'] = billboard['artist'].apply(lambda x: get_featuring(x))
billboard.last_week_rank = billboard['last_week_rank'].astype('float64')
billboard['rank_delta'] = billboard.last_week_rank - billboard['rank']
billboard['year'] = pd.DatetimeIndex(billboard['date_']).year
billboard['month'] = pd.DatetimeIndex(billboard['date_']).month
billboard['week'] = pd.DatetimeIndex(billboard['date_']).week
billboard['unique'] = billboard.artist + '|' + billboard.title
billboard['season'] = billboard['month'].apply(get_Season)

In [15]:
### 10 - Most total weeks on the Hot 100
mwh100 = billboard[billboard['rank']<=100]['unique'].value_counts()[:50]
mwh100a = billboard[billboard['unique'].isin(mwh100.index.values)].drop_duplicates('unique')
mwh100a

,artist,date_,last_week_rank,peak_rank,rank,title,weeks_on_chart,artist1,featuring,rank_delta,year,month,week,unique,season
2720,Ed Sheeran,2018-10-06,23.0,1.0,21,Perfect,57.0,Ed Sheeran,Ed Sheeran,2.0,2018,10,40,Ed Sheeran|Perfect,autumn
4933,Avicii,2018-05-05,NaN,4.0,34,Wake Me Up!,54.0,Avicii,Avicii,NaN,2018,5,18,Avicii|Wake Me Up!,spring
5823,Ed Sheeran,2018-03-03,24.0,1.0,24,Shape Of You,59.0,Ed Sheeran,Ed Sheeran,0.0,2018,3,9,Ed Sheeran|Shape Of You,spring
9938,Justin Timberlake,2017-05-20,38.0,1.0,39,Can't Stop The Feeling!,52.0,Justin Timberlake,Justin Timberlake,-1.0,2017,5,20,Justin Timberlake|Can't Stop The Feeling!,spring
11143,The Chainsmokers Featuring Daya,2017-02-25,36.0,3.0,44,Don't Let Me Down,52.0,The Chainsmokers,Daya,-8.0,2017,2,8,The Chainsmokers Featuring Daya|Don't Let Me Down,winter
16048,Wiz Khalifa Featuring Charlie Puth,2016-03-19,45.0,1.0,49,See You Again,52.0,Wiz Khalifa,Charlie Puth,-4.0,2016,3,11,Wiz Khalifa Featuring Charlie Puth|See You Again,spring
16221,Mark Ronson Featuring Bruno Mars,2016-03-05,NaN,1.0,22,Uptown Funk!,56.0,Mark Ronson,Bruno Mars,NaN,2016,3,9,Mark Ronson Featuring Bruno Mars|Uptown Funk!,spring
17638,WALK THE MOON,2015-11-28,38.0,4.0,39,Shut Up And Dance,53.0,WALK THE MOON,WALK THE MOON,-1.0,2015,11,48,WALK THE MOON|Shut Up And Dance,winter
17644,Ed Sheeran,2015-11-28,41.0,2.0,45,Thinking Out Loud,58.0,Ed Sheeran,Ed Sheeran,-4.0,2015,11,48,Ed Sheeran|Thinking Out Loud,winter
20649,Sam Smith,2015-05-02,49.0,2.0,50,Stay With Me,54.0,Sam Smith,Sam Smith,-1.0,2015,5,18,Sam Smith|Stay With Me,spring


In [16]:
def seasonal_tops(season='winter', top_x = 11, rank=10, peak_rank=5, weeks_on_chart = 3, min_date = '1958-08-16', max_date = '2019-04-13'):
    df_season = billboard.groupby('season').get_group(season)
    temp_season = df_season.loc[df_season['rank'] <= rank, :].loc[df_season['weeks_on_chart'] >= weeks_on_chart, :]
    temp_season_time =  temp_season.loc[df_season['date_'] >= min_date, :].loc[temp_season['date_'] <= max_date, :]
    temp_season_time = temp_season_time[['title', 'artist','unique']] 
    temp_season_time['unique'] = temp_season_time['unique'].str.replace('|', ' - ', regex=False)
#     temp_season_time_cnt = temp_season_time['unique'].value_counts().to_frame().reset_index()
#     temp_season_time_cnt.columns = ['track', 'counts']

#     return temp_season_time_cnt
    return temp_season_time

In [17]:
tops = seasonal_tops(season='winter', rank=30)
tops

,title,artist,unique
700,7 Rings,Ariana Grande,Ariana Grande - 7 Rings
702,"Thank U, Next",Ariana Grande,"Ariana Grande - Thank U, Next"
703,Without Me,Halsey,Halsey - Without Me
704,Sunflower (Spider-Man: Into The Spider-Verse),Post Malone & Swae Lee,Post Malone & Swae Lee - Sunflower (Spider-Man...
705,Happier,Marshmello & Bastille,Marshmello & Bastille - Happier
706,Sicko Mode,Travis Scott,Travis Scott - Sicko Mode
707,High Hopes,Panic! At The Disco,Panic! At The Disco - High Hopes
708,Wow.,Post Malone,Post Malone - Wow.
709,Eastside,"benny blanco, Halsey & Khalid","benny blanco, Halsey & Khalid - Eastside"
710,Middle Child,J. Cole,J. Cole - Middle Child


In [18]:
billboard.drop_duplicates('unique', inplace=True)

In [19]:
client_credentials_manager = SpotifyClientCredentials(client_id=app_credentials.client_id, client_secret=app_credentials.client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_artist(name):
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        return items[0]
    else:
        return None

def get_artist_album_ids(name):
    artist = get_artist(name)
    results = sp.artist_albums(artist['id'], album_type='album', country='US')
    albums = results['items']
    album_ids = []
    album_ids[:] = [album['id'] for album in albums]
    return album_ids

def get_artist_features(name,ftrack,feature='valence'):
    # features: valence (default), acousticness, danceability,
    #           energy, instrumentalness, speechiness, & tempo
    run_once = 0
    d = dict()# initialize dictionary
    name = name_slicer(name)
    album_ids = get_artist_album_ids(name)
    features = []
    for album_id in album_ids:
        results = sp.album_tracks(album_id)
        tracks = results['items']
        for track in tracks:
            if(ftrack.lower() in track['name'].lower()):
                audioFeatures = sp.audio_features(track['id'])
#                 print(track['name']) # track name
#                 print(get_artist(name)['genres']) # ganre
#                 #result = (json_normalize(audioFeatures[0]))
#                 print(audioFeatures[0])
                  #features.append(audioFeatures[0])
                d = dict(zip(['artist','track','genres'],[name,track['name'],get_artist(name)['genres']]))
                d.update(audioFeatures[0])
                return d
            else:
                pass
    return d

In [20]:
# example format of the dictionary
exampleDict = {'artist': 'Ariana Grande',
 'track': 'thank u, next',
 'genres': ['dance pop', 'pop', 'post-teen pop'],
 'danceability': 0.717,
 'energy': 0.653,
 'key': 1,
 'loudness': -5.634,
 'mode': 1,
 'speechiness': 0.0658,
 'acousticness': 0.229,
 'instrumentalness': 0,
 'liveness': 0.101,
 'valence': 0.412,
 'tempo': 106.966,
 'type': 'audio_features',
 'id': '3e9HZxeyfWwjeyPAMmWSSQ',
 'uri': 'spotify:track:3e9HZxeyfWwjeyPAMmWSSQ',
 'track_href': 'https://api.spotify.com/v1/tracks/3e9HZxeyfWwjeyPAMmWSSQ',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3e9HZxeyfWwjeyPAMmWSSQ',
 'duration_ms': 207320,
 'time_signature': 4,
 'unique':'Ariana Grande|thank u, next'}

In [21]:
# An Empty df_spotify
df_spotify = pd.DataFrame(columns=exampleDict.keys())
df_spotify

,artist,track,genres,danceability,energy,key,loudness,mode,speechiness,acousticness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,unique


In [22]:
tt =  billboard.loc[billboard['date_'] >= '2010-01-01', :].loc[billboard['date_'] <= '2019-4-13', :]

In [23]:
songs = tt.drop_duplicates('unique')
songs

,artist,date_,last_week_rank,peak_rank,rank,title,weeks_on_chart,artist1,featuring,rank_delta,year,month,week,unique,season
0,Lil Nas X Featuring Billy Ray Cyrus,2019-04-13,15.0,1.0,1,Old Town Road,5.0,Lil Nas,| Billy Ray Cyrus,14.0,2019,4,15,Lil Nas X Featuring Billy Ray Cyrus|Old Town Road,spring
1,Post Malone & Swae Lee,2019-04-13,3.0,1.0,2,Sunflower (Spider-Man: Into The Spider-Verse),24.0,Post Malone,Swae Lee,1.0,2019,4,15,Post Malone & Swae Lee|Sunflower (Spider-Man: ...,spring
2,Ariana Grande,2019-04-13,1.0,1.0,3,7 Rings,11.0,Ariana Grande,Ariana Grande,-2.0,2019,4,15,Ariana Grande|7 Rings,spring
3,Post Malone,2019-04-13,2.0,2.0,4,Wow.,15.0,Post Malone,Post Malone,-2.0,2019,4,15,Post Malone|Wow.,spring
4,Halsey,2019-04-13,4.0,1.0,5,Without Me,26.0,Halsey,Halsey,-1.0,2019,4,15,Halsey|Without Me,spring
5,Cardi B & Bruno Mars,2019-04-13,5.0,3.0,6,Please Me,7.0,Cardi B,Bruno Mars,-1.0,2019,4,15,Cardi B & Bruno Mars|Please Me,spring
6,Billie Eilish,2019-04-13,NaN,NaN,7,Bad Guy,NaN,Billie Eilish,Billie Eilish,NaN,2019,4,15,Billie Eilish|Bad Guy,spring
7,Jonas Brothers,2019-04-13,9.0,1.0,8,Sucker,5.0,Jonas Brothers,Jonas Brothers,1.0,2019,4,15,Jonas Brothers|Sucker,spring
8,Marshmello & Bastille,2019-04-13,6.0,2.0,9,Happier,33.0,Marshmello,Bastille,-3.0,2019,4,15,Marshmello & Bastille|Happier,spring
9,J. Cole,2019-04-13,7.0,4.0,10,Middle Child,11.0,J. Cole,J. Cole,-3.0,2019,4,15,J. Cole|Middle Child,spring


In [24]:
for (i,j) in zip(songs.artist,songs.title): #getting artist & title
    try:
        a = get_artist_features(i,j)
        if not a:
            pass
        else:
            Series_ = pd.Series(a).append(pd.Series({'unique':i+'|'+j}))
            df_spotify = df_spotify.append(Series_,ignore_index=True)
            k+=1
    except:
        pass

retrying ...1secs


In [25]:
df_spotify


,artist,track,genres,danceability,energy,key,loudness,mode,speechiness,acousticness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,unique
0,Ariana Grande,7 rings,"[dance pop, pop, post-teen pop]",0.778,0.3170,1,-10.732,0,0.3340,0.592000,...,0.3270,140.048,audio_features,6ocbgoVGwYJhOv1GgI9NsF,spotify:track:6ocbgoVGwYJhOv1GgI9NsF,https://api.spotify.com/v1/tracks/6ocbgoVGwYJh...,https://api.spotify.com/v1/audio-analysis/6ocb...,178627,4,Ariana Grande|7 Rings
1,Billie Eilish,bad guy,"[electropop, pop]",0.708,0.4180,0,-10.998,1,0.3680,0.308000,...,0.5780,131.926,audio_features,2Fxmhks0bxGSBdJ92vM42m,spotify:track:2Fxmhks0bxGSBdJ92vM42m,https://api.spotify.com/v1/tracks/2Fxmhks0bxGS...,https://api.spotify.com/v1/audio-analysis/2Fxm...,194088,4,Billie Eilish|Bad Guy
2,Lady Gaga,Shallow,"[dance pop, pop]",0.572,0.3850,7,-6.362,1,0.0308,0.371000,...,0.3230,95.799,audio_features,2VxeLyX666F8uXCJ0dZF8B,spotify:track:2VxeLyX666F8uXCJ0dZF8B,https://api.spotify.com/v1/tracks/2VxeLyX666F8...,https://api.spotify.com/v1/audio-analysis/2Vxe...,215733,4,Lady Gaga & Bradley Cooper|Shallow
3,Blueface,Thotiana,"[cali rap, trap music]",0.906,0.3820,10,-12.890,0,0.2690,0.180000,...,0.3910,104.025,audio_features,3oh6SCCeLuXhFpEyepla6G,spotify:track:3oh6SCCeLuXhFpEyepla6G,https://api.spotify.com/v1/tracks/3oh6SCCeLuXh...,https://api.spotify.com/v1/audio-analysis/3oh6...,129264,4,Blueface|Thotiana
4,Meek Mill,Going Bad (feat. Drake),"[hip hop, philly rap, pop, pop rap, rap, south...",0.889,0.4960,4,-6.365,0,0.0905,0.259000,...,0.5440,86.003,audio_features,2IRZnDFmlqMuOrYOLnZZyc,spotify:track:2IRZnDFmlqMuOrYOLnZZyc,https://api.spotify.com/v1/tracks/2IRZnDFmlqMu...,https://api.spotify.com/v1/audio-analysis/2IRZ...,180522,4,Meek Mill Featuring Drake|Going Bad
5,Khalid,Better,[pop],0.692,0.5520,0,-9.543,0,0.0497,0.050300,...,0.1500,98.028,audio_features,339Y993TrFyWyO438be0RY,spotify:track:339Y993TrFyWyO438be0RY,https://api.spotify.com/v1/tracks/339Y993TrFyW...,https://api.spotify.com/v1/audio-analysis/339Y...,229400,4,Khalid|Better
6,Panic! At The Disco,High Hopes,"[emo, modern rock, pop punk]",0.579,0.9040,5,-2.729,1,0.0618,0.193000,...,0.6810,82.014,audio_features,1rqqCSm0Qe4I9rUvWncaom,spotify:track:1rqqCSm0Qe4I9rUvWncaom,https://api.spotify.com/v1/tracks/1rqqCSm0Qe4I...,https://api.spotify.com/v1/audio-analysis/1rqq...,190947,4,Panic! At The Disco|High Hopes
7,Ariana Grande,"break up with your girlfriend, i'm bored","[dance pop, pop, post-teen pop]",0.726,0.5540,5,-5.290,0,0.0917,0.042100,...,0.3350,169.999,audio_features,4kV4N9D1iKVxx1KLvtTpjS,spotify:track:4kV4N9D1iKVxx1KLvtTpjS,https://api.spotify.com/v1/tracks/4kV4N9D1iKVx...,https://api.spotify.com/v1/audio-analysis/4kV4...,190440,4,"Ariana Grande|Break Up With Your Girlfriend, I..."
8,Travis Scott,SICKO MODE,"[pop, pop rap, rap, trap music]",0.834,0.7300,8,-3.714,1,0.2220,0.005130,...,0.4460,155.008,audio_features,2xLMifQCjDGFmkHkpNLD9h,spotify:track:2xLMifQCjDGFmkHkpNLD9h,https://api.spotify.com/v1/tracks/2xLMifQCjDGF...,https://api.spotify.com/v1/audio-analysis/2xLM...,312820,4,Travis Scott|Sicko Mode
9,YNW Melly,Murder On My Mind,"[rap, trap music]",0.759,0.7300,0,-7.985,0,0.0516,0.145000,...,0.7400,115.007,audio_features,7eBqSVxrzQZtK2mmgRG6lC,spotify:track:7eBqSVxrzQZtK2mmgRG6lC,https://api.spotify.com/v1/tracks/7eBqSVxrzQZt...,https://api.spotify.com/v1/audio-analysis/7eBq...,268434,4,YNW Melly|Murder On My Mind


In [26]:
df_spotify.to_csv("2010s.csv", encoding='utf-8',index=False)

In [20]:
# df_2019.to_csv("spotify_2018-2019.csv", encoding='utf-8',index=False)

In [15]:
# k = 0
# for l, (i,j) in enumerate(zip(billboard.artist,billboard.title)): # getting artist & title
#     if k>2:
#         break
#     else:
#         try:
#             df_spotify.loc[l] = list(get_artist_features(i,j).values())
#             k+=1
#         except:
#             pass

In [ ]:
# k = 0
# for (i,j) in zip(billboard.artist,billboard.title): # getting artist & title
#     if k>2:
#         break
#     else:
#         try:
#             Series_ = pd.Series(get_artist_features(i,j))
#             df_spotify = df_spotify.append(Series_,ignore_index=True) 
#             k+=1
#         except:
#             pass

In [26]:
test_df  = pd.DataFrame(columns=exampleDict.keys())

test_df.loc[1] = list(exampleDict.values())
test_df

,artist,track,genres,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
1,Ariana Grande,"thank u, next","[dance pop, pop, post-teen pop]",0.717,0.653,1,-5.634,1,0.0658,0.229,...,0.101,0.412,106.966,audio_features,3e9HZxeyfWwjeyPAMmWSSQ,spotify:track:3e9HZxeyfWwjeyPAMmWSSQ,https://api.spotify.com/v1/tracks/3e9HZxeyfWwj...,https://api.spotify.com/v1/audio-analysis/3e9H...,207320,4


In [85]:
test_df.loc[1] = np.array(exampleDict.values())

ValueError: setting an array element with a sequence

In [87]:
test_df  = pd.DataFrame(columns=exampleDict.keys())

In [67]:
list(exampleDict.values())

['Ariana Grande',
 'thank u, next',
 ['dance pop', 'pop', 'post-teen pop'],
 0.717,
 0.653,
 1,
 -5.634,
 1,
 0.0658,
 0.229,
 0,
 0.101,
 0.412,
 106.966,
 'audio_features',
 '3e9HZxeyfWwjeyPAMmWSSQ',
 'spotify:track:3e9HZxeyfWwjeyPAMmWSSQ',
 'https://api.spotify.com/v1/tracks/3e9HZxeyfWwjeyPAMmWSSQ',
 'https://api.spotify.com/v1/audio-analysis/3e9HZxeyfWwjeyPAMmWSSQ',
 207320,
 4]

In [71]:
test_df

,artist,track,genres,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Ariana Grande,"thank u, next","[dance pop, pop, post-teen pop]",0.717,0.653,1,-5.634,1,0.0658,0.229,...,0.101,0.412,106.966,audio_features,3e9HZxeyfWwjeyPAMmWSSQ,spotify:track:3e9HZxeyfWwjeyPAMmWSSQ,https://api.spotify.com/v1/tracks/3e9HZxeyfWwj...,https://api.spotify.com/v1/audio-analysis/3e9H...,207320,4
1,Ariana Grande,"thank u, next","[dance pop, pop, post-teen pop]",0.717,0.653,1,-5.634,1,0.0658,0.229,...,0.101,0.412,106.966,audio_features,3e9HZxeyfWwjeyPAMmWSSQ,spotify:track:3e9HZxeyfWwjeyPAMmWSSQ,https://api.spotify.com/v1/tracks/3e9HZxeyfWwj...,https://api.spotify.com/v1/audio-analysis/3e9H...,207320,4


In [44]:
for i in range(len(test_df)):
    for j in range(len(test_series)):
        test_df.loc[i,test_df.columns[j]] = test_series[j]

In [100]:
billboard.artist.count()

299099

In [39]:
for j in range(len(test_series)):
    print(test_series[j])

1
2
3


In [ ]:
k = 0
for (i,j) in zip(billboard.artist,billboard.title): # getting artist & title
    if k>1:
        break
    else:
        try:
            for indexx in range(billboard.artist.count()):
                list_ = list(get_artist_features(i,j).values())
                df_spotify.loc[indexx] = list_
                k+=1
        except:
            pass

From cffi callback <function _verify_callback at 0x000001A782382950>:
Traceback (most recent call last):
  File "c:\users\young min cho\python\python36\lib\site-packages\OpenSSL\SSL.py", line 306, in wrapper
    @wraps(callback)
KeyboardInterrupt
From cffi callback <function _verify_callback at 0x000001A7FBD2F7B8>:
Traceback (most recent call last):
  File "c:\users\young min cho\python\python36\lib\site-packages\OpenSSL\SSL.py", line 306, in wrapper
    @wraps(callback)
KeyboardInterrupt


In [31]:
test_series = pd.Series([1,2,3])

In [32]:
test_series

0    1
1    2
2    3
dtype: int64

In [23]:
test_df.iloc[:,2] = pd.Series([1,2,3])

In [278]:
df_spotify

,artist,track,genres,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Ariana Grande,"thank u, next","[dance pop, pop, post-teen pop]",0.717,0.653,1,-5.634,1,0.0658,0.22900,...,0.1010,0.412,106.966,audio_features,3e9HZxeyfWwjeyPAMmWSSQ,spotify:track:3e9HZxeyfWwjeyPAMmWSSQ,https://api.spotify.com/v1/tracks/3e9HZxeyfWwj...,https://api.spotify.com/v1/audio-analysis/3e9H...,207320,4
1,Travis Scott,SICKO MODE,"[pop, pop rap, rap, trap music]",0.834,0.730,8,-3.714,1,0.2220,0.00513,...,0.1240,0.446,155.008,audio_features,2xLMifQCjDGFmkHkpNLD9h,spotify:track:2xLMifQCjDGFmkHkpNLD9h,https://api.spotify.com/v1/tracks/2xLMifQCjDGF...,https://api.spotify.com/v1/audio-analysis/2xLM...,312820,4
2,Panic! At The Disco,High Hopes,"[emo, modern rock, pop punk]",0.579,0.904,5,-2.729,1,0.0618,0.19300,...,0.0640,0.681,82.014,audio_features,1rqqCSm0Qe4I9rUvWncaom,spotify:track:1rqqCSm0Qe4I9rUvWncaom,https://api.spotify.com/v1/tracks/1rqqCSm0Qe4I...,https://api.spotify.com/v1/audio-analysis/1rqq...,190947,4
3,Maroon 5,Girls Like You,[pop],0.611,0.462,0,-7.054,1,0.0646,0.55800,...,0.1310,0.377,59.635,audio_features,6OEHjJJvdkk1Ln0p0AZdyz,spotify:track:6OEHjJJvdkk1Ln0p0AZdyz,https://api.spotify.com/v1/tracks/6OEHjJJvdkk1...,https://api.spotify.com/v1/audio-analysis/6OEH...,215230,4
4,Kodak Black,ZEZE (feat. Travis Scott & Offset),"[miami hip hop, rap, trap music]",0.861,0.603,8,-5.788,0,0.1760,0.05210,...,0.0924,0.504,98.043,audio_features,0FZ4Dmg8jJJAPJnvBIzD9z,spotify:track:0FZ4Dmg8jJJAPJnvBIzD9z,https://api.spotify.com/v1/tracks/0FZ4Dmg8jJJA...,https://api.spotify.com/v1/audio-analysis/0FZ4...,228760,4
5,Post Malone,Better Now,"[pop, rap]",0.680,0.563,10,-5.843,1,0.0454,0.35400,...,0.1360,0.374,145.028,audio_features,7dt6x5M1jzdTEt8oCbisTK,spotify:track:7dt6x5M1jzdTEt8oCbisTK,https://api.spotify.com/v1/tracks/7dt6x5M1jzdT...,https://api.spotify.com/v1/audio-analysis/7dt6...,231267,4


In [138]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import pprint
import app_credentials
import json
from pandas.io.json import json_normalize

client_credentials_manager = SpotifyClientCredentials(client_id=app_credentials.client_id, client_secret=app_credentials.client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


def get_artist(name):
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        return items[0]
    else:
        return None

def get_artist_album_ids(name):
    artist = get_artist(name)
    results = sp.artist_albums(artist['id'], album_type='album', country='US')
    albums = results['items']
    album_ids = []
    for album in albums:
        album_ids.append(album['id'])

    return album_ids

def get_artist_features(name, feature='valence'):
    # features: valence (default), acousticness, danceability,
    #           energy, instrumentalness, speechiness, & tempo
    if 'Featuring' in name:
        name = name[:name.find("Featuring")-1]
    album_ids = get_artist_album_ids(name)
    features = []
    for album_id in album_ids:
        results = sp.album_tracks(album_id)
        tracks = results['items']
        for track in tracks:
            audioFeatures = sp.audio_features(track['id'])
            print(track['name'])
            #print(json_normalize(audioFeatures))
            features.append(audioFeatures[0])

            
            #track['name']

            
    return None #features

In [139]:
get_artist_features("Ross Lynch")

In [76]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import pprint
import app_credentials
import json
from pandas.io.json import json_normalize

client_credentials_manager = SpotifyClientCredentials(client_id=app_credentials.client_id, client_secret=app_credentials.client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


def get_artist(name):
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        return items[0]
    else:
        return None

def get_artist_album_ids(name):
    artist = get_artist(name)
    results = sp.artist_albums(artist['id'], album_type='album', country='US')
    albums = results['items']
    album_ids = []
    for album in albums:
        album_ids.append(album['id'])

    return album_ids

def get_artist_features(name,ftrack,feature='valence'):
    # features: valence (default), acousticness, danceability,
    #           energy, instrumentalness, speechiness, & tempo
    run_once = 0
    if 'Featuring' in name:
        name = name[:name.find("Featuring")-1]
    album_ids = get_artist_album_ids(name)
    features = []
    for album_id in album_ids:
        results = sp.album_tracks(album_id)
        tracks = results['items']
        for track in tracks:
            if(ftrack.lower() in track['name'].lower() and run_once==0):
                audioFeatures = sp.audio_features(track['id'])
#                 print(track['name']) # track name
#                 print(get_artist(name)['genres']) # ganre
#                 #result = (json_normalize(audioFeatures[0]))
#                 print(audioFeatures[0])
                  #features.append(audioFeatures[0])
                d = dict() # initialize dictionary
                d['artist'] = name
                d['track'] = track['name']
                d['genres'] = get_artist(name)['genres']
                d.update(audioFeatures[0])

                run_once += 1 # run once
            else:
                pass
            

            
    return d

In [49]:
test_df.loc[0,'a']=1

In [51]:
test_df.loc[0] = [1,2,3]

In [69]:
test_df

,artist,track,genres,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Ariana Grande,"thank u, next","[dance pop, pop, post-teen pop]",0.717,0.653,1,-5.634,1,0.0658,0.229,...,0.101,0.412,106.966,audio_features,3e9HZxeyfWwjeyPAMmWSSQ,spotify:track:3e9HZxeyfWwjeyPAMmWSSQ,https://api.spotify.com/v1/tracks/3e9HZxeyfWwj...,https://api.spotify.com/v1/audio-analysis/3e9H...,207320,4


In [10]:
get_artist_features("Tory Lanez","Talk to me")

{'artist': 'Tory Lanez',
 'track': 'TAlk tO Me (with Rich The Kid)',
 'genres': ['canadian hip hop',
  'hip hop',
  'pop',
  'pop rap',
  'r&b',
  'rap',
  'southern hip hop',
  'trap music'],
 'danceability': 0.731,
 'energy': 0.657,
 'key': 0,
 'loudness': -8.56,
 'mode': 1,
 'speechiness': 0.0434,
 'acousticness': 0.0591,
 'instrumentalness': 0,
 'liveness': 0.0874,
 'valence': 0.49,
 'tempo': 159.97,
 'type': 'audio_features',
 'id': '2cRgo1TiPm7SnPvYzvbOLf',
 'uri': 'spotify:track:2cRgo1TiPm7SnPvYzvbOLf',
 'track_href': 'https://api.spotify.com/v1/tracks/2cRgo1TiPm7SnPvYzvbOLf',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2cRgo1TiPm7SnPvYzvbOLf',
 'duration_ms': 184490,
 'time_signature': 4}

In [237]:
def list_to_list(row):
    return row.list

In [240]:
a = [11,2,3,4,5]
a.

AttributeError: 'builtin_function_or_method' object has no attribute 'pop'